In [1]:
import pandas as pd

# Use absolute path temporarily
csv_path = r"C:\Users\Pc\Desktop\Emotion_Music_Recommender\Data\Deam\annotations_45s.csv\annotations\annotations averaged per song\song_level\static_annotations_averaged_songs_1_2000.csv"

# Try loading
df_deam = pd.read_csv(csv_path)

# Preview
df_deam.head()

,song_id,valence_mean,valence_std,arousal_mean,arousal_std
0,2,3.1,0.94,3.0,0.63
1,3,3.5,1.75,3.3,1.62
2,4,5.7,1.42,5.5,1.63
3,5,4.4,2.01,5.3,1.85
4,7,5.8,1.47,6.4,1.69


In [2]:
# Clean up column names by stripping leading/trailing whitespace
df_deam.columns = df_deam.columns.str.strip()

In [3]:
# Step 2: Convert valence and arousal to emotion labels
def get_emotion(valence, arousal):
    if valence < 5 and arousal < 5:
        return "sadness"
    elif valence < 5 and arousal >= 5:
        return "anger"
    elif valence >= 5 and arousal >= 5:
        return "joy"
    elif valence >= 5 and arousal < 5:
        return "calm"
    return "neutral"

# Apply emotion classification
df_deam['emotion'] = df_deam.apply(lambda row: get_emotion(row['valence_mean'], row['arousal_mean']), axis=1)

# View the first few with emotion
df_deam[['song_id', 'valence_mean', 'arousal_mean', 'emotion']].head()

,song_id,valence_mean,arousal_mean,emotion
0,2,3.1,3.0,sadness
1,3,3.5,3.3,sadness
2,4,5.7,5.5,joy
3,5,4.4,5.3,anger
4,7,5.8,6.4,joy


In [4]:
# Function to recommend songs by emotion
def recommend_songs_by_emotion(emotion, n=5):
    filtered = df_deam[df_deam['emotion'] == emotion]
    if filtered.empty:
        print(f"No songs found for emotion: {emotion}")
        return pd.DataFrame()
    return filtered.sample(n)

# Example: Recommend 5 joyful songs
recommend_songs_by_emotion("joy", n=5)

,song_id,valence_mean,valence_std,arousal_mean,arousal_std,emotion
277,327,6.0,1.34,7.1,1.30,joy
1142,1399,6.1,1.70,6.4,1.62,joy
1317,1574,5.9,1.64,5.4,1.69,joy
444,556,5.6,2.06,6.1,1.64,joy
1287,1544,6.4,1.74,5.4,1.62,joy


In [5]:
import os

# Updated path to the 45-second audio clips
audio_dir = r"C:\Users\Pc\Desktop\Emotion_Music_Recommender\Data\Deam\audio_45_sec\MEMD_audio"

# Function to convert song_id (int) → audio filename
def format_song_id(song_id):
    return f"{int(song_id):04d}.mp3"

# Add full audio path to each song
df_deam["audio_path"] = df_deam["song_id"].apply(lambda sid: os.path.join(audio_dir, format_song_id(sid)))

# Preview
df_deam[["song_id", "emotion", "audio_path"]].head()

,song_id,emotion,audio_path
0,2,sadness,C:\Users\Pc\Desktop\Emotion_Music_Recommender\...
1,3,sadness,C:\Users\Pc\Desktop\Emotion_Music_Recommender\...
2,4,joy,C:\Users\Pc\Desktop\Emotion_Music_Recommender\...
3,5,anger,C:\Users\Pc\Desktop\Emotion_Music_Recommender\...
4,7,joy,C:\Users\Pc\Desktop\Emotion_Music_Recommender\...


In [7]:
from IPython.display import Audio

# Get 1 song with desired emotion
song = recommend_songs_by_emotion("calm", n=1).iloc[0]

# Show info
print(f"Playing song ID: {song['song_id']} ({song['emotion']})")

# Play audio
Audio(filename=song["audio_path"])

Playing song ID: 1503 (calm)


In [8]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from textblob import TextBlob

In [10]:
def text_to_emotion(text):
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    subjectivity = analysis.sentiment.subjectivity

    # Basic mapping based on polarity
    if polarity >= 0.5:
        return "joy"
    elif polarity >= 0.1:
        return "calm"
    elif polarity <= -0.5:
        return "anger"
    else:
        return "sadness"

In [11]:
def recommend_from_text(text, n=3):
    emotion = text_to_emotion(text)
    print(f"Detected Emotion: {emotion}")
    songs = recommend_songs_by_emotion(emotion, n)
    return songs

In [12]:
import os

# Updated function that checks for existing audio files
def recommend_songs_by_emotion(emotion, n=5):
    filtered = df_deam[df_deam['emotion'] == emotion].copy()
    
    # Only keep rows where the audio file exists
    filtered = filtered[filtered['audio_path'].apply(os.path.exists)]

    if filtered.empty:
        print(f"No available audio files for emotion: {emotion}")
        return pd.DataFrame()

    return filtered.sample(n)

In [13]:
from IPython.display import Audio, display

user_input = "I’m feeling tired and bored"
songs = recommend_from_text(user_input, n=2)

if not songs.empty:
    display(songs[['song_id', 'emotion']])
    display(Audio(filename=songs.iloc[0]['audio_path']))

Detected Emotion: sadness


,song_id,emotion
1506,1763,sadness
754,1011,sadness


In [14]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import os
from textblob import TextBlob
from IPython.display import display, Audio
import ipywidgets as widgets

# Step 1: Text → Emotion
def text_to_emotion(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity >= 0.5:
        return "joy"
    elif polarity >= 0.1:
        return "calm"
    elif polarity <= -0.5:
        return "anger"
    else:
        return "sadness"

# Step 2: Recommend songs with existing audio
def recommend_songs_by_emotion(emotion, n=5):
    filtered = df_deam[df_deam['emotion'] == emotion].copy()
    filtered = filtered[filtered['audio_path'].apply(os.path.exists)]
    return filtered.sample(n) if not filtered.empty else pd.DataFrame()

# Step 3: Full Recommendation UI
def recommend_and_play(text):
    emotion = text_to_emotion(text)
    print(f"🧠 Detected Emotion: {emotion}")

    songs = recommend_songs_by_emotion(emotion, n=5)

    if songs.empty:
        print("⚠️ No matching audio files found.")
        return

    dropdown = widgets.Dropdown(
        options=[(f"Song ID: {row['song_id']} | Emotion: {row['emotion']}", row['audio_path']) for _, row in songs.iterrows()],
        description='🎵 Songs:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='600px')
    )

    output = widgets.Output()

    def on_select(change):
        output.clear_output()
        with output:
            display(Audio(filename=change.new))

    dropdown.observe(on_select, names='value')

    display(dropdown, output)


In [17]:
recommend_and_play("I feel relaxed and peaceful")

🧠 Detected Emotion: calm


Dropdown(description='🎵 Songs:', layout=Layout(width='600px'), options=(('Song ID: 1791 | Emotion: calm', 'C:\…

Output()